# PyAVD - Quick Solver

In [1]:
from gpkit import Model, ureg
from gpkit.interactive.sankey import Sankey
from gpkit.constraints.bounded import Bounded
from gpkit.interactive.plotting import treemap
from Models import *
# %matplotlib inline

In [2]:
# with gpkit.SignomialsEnabled():

#     AC = Aircraft()
#     # print(AC)

#     MISSION = Mission(AC)
#     # print(MISSION)

#     M = Model(AC.M_0 * AC.T0_W0, [Bounded(MISSION), Bounded(AC)])
#     print(M)

#     print(M.variables_byname('AR'))

#     sol = M.localsolve(verbosity=1)
#     print(sol.table())

#     print(M.program.results)

In [3]:

AC = Aircraft()
# print(AC)

MISSION = Mission(AC)
# print(MISSION)

# M = Model(AC.M_0 * AC.T0_W0, [Bounded(MISSION), Bounded(AC)])
# M = Model(AC.T0_W0 * AC.M_0, [MISSION, AC])
M = Model(AC.M_fuel, [MISSION, AC])
print(M)



Cost Function
-------------
 M_fuel

Constraints
-----------
 Mission
  "Top-level constraints":
    M[0] ≥ M_fuel + Aircraft.Payload.M + Aircraft.Port_Engine.M + Aircraft.Starboard_Engine.M
    M[1] ≤ M[0]
    M[2] ≤ M[1]
    M[3] ≤ M[2]
    M[4] ≤ M[3]
    M[5] ≤ M[4]
    M[6] ≤ M[5]
    M[6] ≥ Aircraft.Payload.M + Aircraft.Port_Engine.M + Aircraft.Starboard_Engine.M
    M[6] ≥ (Aircraft.Payload.M + Aircraft.Port_Engine.M + Aircraft.Starboard_Engine.M)·1.06
  Takeoff
   "Takeoff Parameter":
     TOP = Mission.Takeoff.FL/Mission.Takeoff.k
   "Lift Coeffcient | Takeoff":
     CL_max_TO = 1.92
   "Thrust to Weight constraint":
     T0_W0 ≥ W0_S/(CL_max_TO·Mission.Takeoff.g·TOP/1.21)
   "Fuel Fraction | Takeoff":
     M[1]/M[0] = 0.97
   "Boundaries":
     "Minimum Wing Loading":
       W0_S ≥ 0.1
     "Maximum Thrust to Weight":
       T0_W0 ≤ 1

  Climb
   "Initial CL|Climb":
     Mission.Climb.CL = 1.92
   "Induced Drag Coefficient | Climb":
     Mission.Climb.CDi_climb = Mission.Cli

In [4]:
fig = treemap(M)
fig

In [5]:
sol = M.solve(verbosity=1)
print(sol.table())

Using solver 'cvxopt'
 for 28 free variables
  in 64 posynomial inequalities.
Solving took 0.244 seconds.

Optimal Cost
------------
 10

~~~~~~~~
WARNINGS
~~~~~~~~
Unexpectedly Loose Constraints
------------------------------
               | Aircraft.Payload
   1725 >= 708 : .M ≥ M_pax + M_luggage

               | Mission.Cruise1
2.538 >= 1.199 : M[4]/M[5] ≥ 1 + (.R·Aircraft.Starboard_Engine.sfc_cruise/(.V_inf·.LD))^1
                   + (.R·Aircraft.Starboard_Engine.sfc_cruise/(.V_inf·.LD))²/2
                   + (.R·Aircraft.Starboard_Engine.sfc_cruise/(.V_inf·.LD))³/6
                   + (.R·Aircraft.Starboard_Engine.sfc_cruise/(.V_inf·.LD))⁴/24
                   + (.R·Aircraft.Starboard_Engine.sfc_cruise/(.V_inf·.LD))⁵/120
                   + (.R·Aircraft.Starboard_Engine.sfc_cruise/(.V_inf·.LD))⁶/720
                   + (.R·Aircraft.Starboard_Engine.sfc_cruise/(.V_inf·.LD))⁷/5.04e+03
                   + (.R·Aircraft.Starboard_Engine.sfc_cruise/(.V_inf·.LD))⁸/4.03e+04
   

In [6]:
from gpkit.interactive.references import referencesplot
referencesplot(M, openimmediately=False)

In [7]:
Sankey(sol, M)

In [8]:
# from functools import reduce

# ln_breguet = 0.181818181818

# 1/1+reduce(lambda x,y: x+y, [ln_breguet**i/np.math.factorial(i) for i in range(1, 5)])